In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM mapping_categories WHERE categorie IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
PCLAT,6219,PERCHLORATE,Perchlorate,14797-73-0,perchlorate


In [3]:
%%sql
SELECT * FROM mapping_categories
WHERE libminparametre IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
14DAN,1580,"1,4 DIOXANE","1,4 dioxane",123-91-1,hydrocarbures


In [4]:
%%sql
SELECT DISTINCT 
    cdparametresiseeaux, limitequal_float , limitequal
FROM int__resultats_udi_communes
WHERE cdparametresiseeaux IN ('14DAN','PCLAT')

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdparametresiseeaux,limitequal_float,limitequal
PCLAT,None,None
14DAN,None,None
